In [ ]:
%reset
%load_ext autoreload
%autoreload 2

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../../../")

import torch
import numpy as np
import matplotlib.pyplot as plt

import implicitmodules.torch as dm

torch.manual_seed(1337)
torch.set_default_tensor_type(torch.DoubleTensor)

In [ ]:
d = 2

nb_pts_trans = 10
nb_pts_trans2 = 5
nb_pts_silent = 25

gd_trans = torch.rand(nb_pts_trans, d, requires_grad=True).view(-1)
gd_trans2 = torch.rand(nb_pts_trans2, d, requires_grad=True).view(-1)
gd_silent = torch.rand(nb_pts_silent, d, requires_grad=True).view(-1)

mom_trans = torch.rand(nb_pts_trans, d, requires_grad=True).view(-1)
mom_trans2 = torch.rand(nb_pts_trans2, d, requires_grad=True).view(-1)
mom_silent = torch.rand(nb_pts_silent, d, requires_grad=True).view(-1)

man_trans = dm.manifold.Landmarks(d, nb_pts_trans, gd=gd_trans, cotan=mom_trans)
man_trans2 = dm.manifold.Landmarks(d, nb_pts_trans2, gd=gd_trans2, cotan=mom_trans2)
man_silent = dm.manifold.Landmarks(d, nb_pts_silent, gd=gd_silent, cotan=mom_silent)

In [ ]:
sigma = 0.1

trans = dm.deformationmodules.Translations(man_trans, sigma)
trans2 = dm.deformationmodules.Translations(man_trans2, sigma)
silent = dm.deformationmodules.SilentPoints(man_silent)

compound = dm.deformationmodules.CompoundModule([trans, trans2, silent])

In [ ]:
hamiltonian = dm.hamiltonian.Hamiltonian(compound)

In [ ]:
out = dm.shooting.shoot(hamiltonian, 10, "torch_euler")

In [ ]:
nx, ny = 100, 100
sx, sy = 10, 10
x, y = torch.meshgrid([torch.arange(0, nx), torch.arange(0, ny)])
x = sx*(x.type(torch.get_default_dtype())/nx - 0.5)
y = sy*(y.type(torch.get_default_dtype())/ny - 0.5)
u, v = dm.usefulfunctions.vec2grid(compound(dm.usefulfunctions.grid2vec(x, y)), nx, ny)

plt.quiver(x.numpy(), y.numpy(), u.detach().numpy(), v.detach().numpy())
plt.show()